In [ ]:
from torch import from_numpy as toTensor, svd as __svd, qr as __qr, einsum as t_einsum, \
    pinverse as __pinv, div as divide, transpose as __transpose, Tensor as typeTensor, \
    stack as __stack
from functools import wraps      
from numpy import finfo as np_finfo, einsum as np_einsum
from numpy import float32 as np_float32, float64 as np_float64, int32 as np_int32, int64 as np_int64
from scipy.linalg.lapack import clapack
from scipy.stats import t as tdist
use_numpy_einsum = True

def Tensor(*args):
    out = []
    for x in args:
        if type(x) is not typeTensor:
            out.append(  toTensor(x)  )
        else:
            out.append(  x  )
    return out

def Numpy(*args):
    out = []
    for x in args:
        if type(x) is typeTensor:
            out.append(  x.numpy()  )
        else:
            out.append(  x  )
    return out

def return_numpy(*args):
    result = Numpy(*args)
    if len(result) == 1:
        return result[0]
    else:
        return tuple(result)

def n2n(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        returned = f(*Tensor(*args), **kwargs)
        
        if type(returned) not in (tuple, list):
            returned = [returned]
        
        return return_numpy(*returned)
    return wrapper

def n2t(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        returned = f(*Tensor(*args), **kwargs)
        return returned
    return wrapper

def einsum(notation, *args, tensor = False):
    if use_numpy_einsum:
        args = Numpy(*args)
        out = np_einsum(notation, *args)
    else:
        args = Tensor(*args)
        try:
            out = t_einsum(notation, *args)
        except:
            out = t_einsum(notation, args)
    if tensor:
        return toTensor(out)
    return out

def T(X):
    if type(X) is not typeTensor:
        X = toTensor(X)
    if len(X.shape) == 1:
        return X.reshape(-1,1)
    return __transpose(X, 0, 1)

def ravel(y, X):
    return Tensor(y.ravel().astype( dtype(X) ))

def constant(X):
    return X.item()

def eps(X):
    try:
        return np_finfo(dtype(X)).eps
    except:
        return np_finfo(np_float64).eps
    
def dtype(tensor):
    if 'float32' in str(tensor.dtype): return np_float32
    elif 'float64' in str(tensor.dtype): return np_float64
    elif 'int32' in str(tensor.dtype): return np_int32
    elif 'int64' in str(tensor.dtype): return np_int64
    else: return np_float32
    
def stack(*args):
    if type(args[0]) is list:
        args = args[0]
    toStack = Tensor(*args)
    return __stack(toStack)

In [ ]:
def t_svd(X):
    U, S, V = __svd(X, some = True)
    return U, S, T(V)
svd = n2n(t_svd)
_svd = n2t(t_svd)


def t_pinv(X):
    U, S, VT = _svd(X)
    cond = S < eps(X)*constant(S[0])
    _S = 1.0 / S
    _S[cond] = 0.0
    VT *= T(_S)
    return T(VT).matmul(T(U))
pinv = n2n(t_pinv)
_pinv = n2t(t_pinv)


def t_qr(X):
    return __qr(X)
qr = n2n(t_qr)
_qr = n2t(t_qr)


def qr_solve(X, y):
    '''
    theta =  R^-1 * QT * y
    '''
    Q, R = qr(X)
    check = 0
    if R.shape[0] == R.shape[1]:
        _R, check = clapack.strtri(R)
    if check > 0:
        _R = _pinv(R)
    Q, _R, R = Tensor(Q, _R, R)
    
    theta_hat = _R.matmul(   T(Q).matmul( ravel(y, Q) )   )
    return theta_hat


def svd_solve(X, y):
    '''
    theta =  V * S^-1 * UT * y
    '''
    U, S, VT = _svd(X)
    cond = S < eps(X)*constant(S[0])
    _S = 1.0 / S
    _S[cond] = 0.0
    VT *= T(_S)
    
    theta_hat = T(VT).matmul(  
                            T(U).matmul(  ravel(y, U)  )
                            )
    return theta_hat


def ridge_solve(X, y, alpha = 1):
    '''
                    S
    theta =   V --------- UT y 
                 S^2 + aI
    '''
    U, S, VT = _svd(X)
    cond = S < eps(X)*constant(S[0])
    _S = S / (S**2 + alpha)
    _S[cond] = 0.0
    VT *= T(_S)
    
    theta_hat = T(VT).matmul(  
                            T(U).matmul(  ravel(y, U)  )
                            )
    return theta_hat


def qr_stats(Q, R):
    '''
    XTX^-1  =  RT * R
    
    h = diag  Q * QT
    
    mean(h) used for normalized leverage
    '''
    XTX = T(R).matmul(R)
    _XTX = pinv(XTX)
    ## Einsum is slow in pytorch so revert to numpy version
    h = einsum('ij,ij->i', Q, Q )
    h_mean = h.mean()
    
    return _XTX, h, h_mean


def svd_stats(U, S, VT):
    '''
                  1
    XTX^-1 =  V ----- VT 
                 S^2
    
    h = diag U * UT
    
    mean(h) used for normalized leverage
    '''
    _S2 = 1.0 / (S**2)
    VS = T(VT) * _S2
    _XTX = VS.matmul(VT)
    h = einsum('ij,ij->i', U, U )
    h_mean = h.mean()
    
    return _XTX, h, h_mean


def ridge_stats(U, S, VT, alpha = 1):
    '''
                               S^2
    exp_theta_hat =  diag V --------- VT
                            S^2 + aI
                            
                                 S^2
    var_theta_hat =  diag V ------------- VT
                            (S^2 + aI)^2
    
                    1
    XTX^-1 =  V --------- VT
                S^2 + aI
                
                  S^2
    h = diag U --------- UT
                S^2 + aI
    
    mean(h) used for normalized leverage
    '''
    V = T(VT)
    S2 = S**2
    S2_alpha = S2 + alpha
    S2_over_S2 = S2 / S2_alpha
    
    VS = V * S2_over_S2
    exp_theta_hat = einsum('ij,ji->i', VS, VT )     # Same as VS.dot(VT)
    
    V_S2 = VS / S2_alpha  # np_divide(S2,  np_square( S2 + alpha ) )
    var_theta_hat = einsum('ij,ij->i',  V_S2  , V )   # Sams as np_multiply(   V,   V_S2 ).sum(1)
    
    _XTX = (V * (1.0 / S2_alpha )  ).matmul( VT )   # V  1/S^2 + a  VT
    
    h = einsum('ij,ij->i', (U * S2_over_S2), U )  # Same as np_multiply(  U*S2_over_S2, U ).sum(1)
    h_mean = h.mean()
    
    return exp_theta_hat, var_theta_hat, _XTX, h_mean

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
torch

In [2]:
import numpy as np
from hyperlearn.hyperlearn.linalg import *
from hyperlearn.hyperlearn.solvers import *
from hyperlearn.hyperlearn.base import *
from sklearn.datasets import make_classification, make_regression
import torch
import scipy.linalg as linalg
n = 100000
p = 200
#X, y = make_classification(random_state = 0, n_samples = int(n/2), n_features = p, n_classes = 10,
#                          n_informative = int(p/3))

X, y = make_regression(random_state = 0, n_samples = n, n_features = p, n_informative = int(p/3))

X[0,0] = 0.0
X[1,1] = 1.0
X[:int(n/2),2] = 0
X[int(n/2):,2] = 1
X[:int(n/2),3] = 0
X[int(n/2):,3] = 1
X[:int(n/2),4] = 1
X[int(n/2):,4] = 0

for x in range(10,50):
    X[:int(n/2),x] = 1
    X[int(n/2):,x] = 2
    
X = X.astype(np.float32, copy = False)
y = y.astype(np.float32, copy = False)
results = [n,p]
# y = y.astype(np.str)

def eigh(X):
    return linalg.eigh(X, b = np.eye(len(X), dtype = X.dtype), turbo = True,
                 check_finite = False)

In [327]:
U, S ,VT = linalg.svd(X, full_matrices = False)

In [340]:
(eigh(XTX)[1][:,::-1] @ VT).round()

array([[-1., -0., -0., ...,  0., -0., -0.],
       [-0., -0., -0., ...,  0.,  0., -0.],
       [-0., -0., -0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ..., -1.,  0., -0.],
       [-0.,  0.,  0., ...,  0., -1.,  0.],
       [-0., -0.,  0., ..., -0.,  0., -1.]], dtype=float32)

In [323]:
from scipy.sparse.linalg import eigsh, svds
eigsh(A = XTX, k = 3, which = 'LM')[1] @ svds(A = X, k = 3, which = 'LM')[2].T

ValueError: shapes (200,3) and (200,3) not aligned: 3 (dim 1) != 200 (dim 0)

In [ ]:
%%time
import fbpca 
from scipy.sparse.linalg import svds
print(round(fbpca.diffsnorm(X, *svds(X, k = 6)), 3))

In [ ]:
# %%time
# round(fbpca.diffsnorm(X, *np.linalg.svd(X, full_matrices = False)), 3)

In [ ]:
%%time
print(round(fbpca.diffsnorm(X, *fbpca.pca(X, k = 6)), 3))

In [ ]:
%%time
X.dot(choleskySolve(X, y, alpha = 0.1))

In [ ]:
# %%time
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept = False, normalize = False, n_jobs = -1)
model.fit(X, y)
squareSum(y - model.predict(X))

In [ ]:
import tensorflow as tf
tf.enable_eager_execution()
from tensorflow.linalg import lstsq


In [14]:
%%time
soln = torch.gels(Tensor(y).reshape(-1,1), Tensor(X))[0][:X.shape[1]].flatten()
print(squareSum(y - X @ soln))

0.004827002
Wall time: 921 ms


In [ ]:
Q, R = linalg.qr(X.T.dot(X) + diagonal(p, 1, X.dtype), mode = 'economic', check_finite = False)

In [ ]:
%%time
preds = X @ (dtrtri(R)[0] @ (Q.T @ (X.T @ y)))

In [ ]:
(dtrtri(R))

In [ ]:
squareSum(y - X @ choleskySolve(X, y))

In [ ]:
from scipy.linalg.lapack import dtrtri
np.set_printoptions(suppress = True, floatmode = 'fixed', precision = 4)

In [ ]:
%%time
einsum('i,j->', dtrtri(R)[0].dot(R))

In [ ]:
%%time
linalg.solve_triangular(R, dot(Q.T, XTy))

In [ ]:
diagSum(Tensor(dtrtri(R)[0]), Tensor(R), transpose_a = True)

In [ ]:
einsum('ji,ij->', dtrtri(R)[0] , R )

In [ ]:
%%time
j = Tensor(X).t()
j @= Tensor(X)

In [ ]:
XTX = cov(X)
XTy = X.T @ y
Q, R = linalg.qr(XTX + diagonal(p, 1, X.dtype) , mode ='economic')


In [ ]:
((y - X.dot(choleskySolve(X, y)))**2).sum()

In [ ]:
%%time
alpha = 0
use_gpu = True
XTX = cov(X)
p = XTX.shape[0]
alpha = np_finfo(X.dtype).resolution if alpha == 0 else alpha
regularizer = diagonal(p, 1, X.dtype)

no_success = True
warn = False

while no_success:
    alphaI = regularizer*alpha
    try:
        if use_gpu: chol = cholesky(  XTX + alphaI  )
        else: chol = linalg.cholesky(  XTX + alphaI  , check_finite = False)
        no_success = False
    except:
        alpha *= 10
        print(alpha)
        warn = True

In [ ]:
%%time
import scipy.linalg as linalg
U, S, VT = linalg.svd(X, full_matrices = False, check_finite = False)

In [ ]:
S.round(2)[:10], Sa.round(2)[:10]

In [ ]:
U.dot(np.diag(Sa)).dot(VTa).round(2), X.round(2)

In [ ]:
%%time

U2, S2, VT2 = linalg.svd(X.T.dot(X), full_matrices = True, check_finite = False)

In [ ]:
VT2.round(2)

In [ ]:
%%time
lambda_, V = linalg.eigh(X.T.dot(X), check_finite  = False)

In [ ]:
Q, R = torch.qr(Tensor(X))

In [ ]:
%%time
U2, S2, VT2 = torch.svd(Tensor(X.T.dot(X)))

In [ ]:
%%time
inv = torch.pinverse(Tensor(X.T.dot(X)))

In [ ]:
%%time
inv = linalg.pinvh(X.T.dot(X) + np.diag(np.ones(p)*0.001), check_finite = False, return_rank = False)

In [ ]:
linalg.svd

In [2]:
%%time
from numpy.linalg import svd as np_svd, lstsq as np_lstsq, qr as np_qr, pinv as np_pinv \
                    , eigh as np_eigh
from numpy import finfo as np_finfo, divide as np_divide, dot as np_dot, multiply as np_multiply, \
                    einsum, square as np_square, newaxis as np_newaxis, log as np_log, sqrt as np_sqrt, \
                    arange as np_arange, array as np_array, argmax as np_argmax, sign as np_sign, \
                    abs as np_abs
from numba.types import Tuple as _Tuple, float32, float64, int32, int64, Array, UniTuple
from numba import njit, jit
from scipy.stats import t as tdist
import numpy as np

matrix32 = float32[:,:]
matrix64 = float64[:,:]
array32 = float32[:]
array64 = float64[:]
int32A = int32[:]
int64A = int64[:]

def column(a): return a[:,np_newaxis]
def row(a): return a[np_newaxism,:]

# @njit( [ Tuple((matrix32, array32, matrix32))  (matrix32) ,
#          Tuple((matrix64, array64, matrix64))  (matrix64) ] , fastmath = True, nogil = True)
# def ___svd(X):
#     return np_svd(X, full_matrices = False)

# @njit( [ UniTuple(matrix32, 2)  (matrix32) ,
#          UniTuple(matrix64, 2)  (matrix64) ] , fastmath = True, nogil = True)
# def _qr(X):
#     return np_qr(X)

# @njit( [ matrix32(matrix32) , matrix64(matrix64) ] , fastmath = True, nogil = True)
# def ___pinv(X):
#     U, S, VT = np_svd(X, full_matrices = False)
#     cond =  S < np_finfo(X.dtype).eps*S[0] 
#     S = 1/S
#     S[cond] = 0.0
#     VT *= S.reshape(-1,1)
#     return VT.T @ U.T

# @njit( [ matrix32(matrix32) , matrix64(matrix64) ] , fastmath = True, nogil = True)
# def __pinv(X):
#     return np_pinv(X)


from numba import f4, f8

def Tuple(*args):
    return _Tuple(args)

Wall time: 276 ms


In [3]:
@njit(['f4[:,:](f4[:,:])','f8[:,:](f8[:,:])'] , fastmath = True, nogil = True, cache = True)
def ___pinv(X):
    U, S, VT = np_svd(X, full_matrices = False)
    cond =  S < np_finfo(X.dtype).eps*S[0]
    S = 1/S
    S[cond] = 0.0
    VT *= S.reshape(-1,1)
    return VT.T @ U.T


# @njit( ['f4[:](f4[:,:], f4[:])','f8[:](f8[:,:], f8[:])'] , fastmath = True, nogil = True, cache = True)
# def ___lstsq(X, y):
#     return np_lstsq(X, y.astype(X.dtype))[0]


# @njit( [ Tuple((f4[:], f4[:,:]))(f4[:,:]) , Tuple((f8[:], f8[:,:]))(f8[:,:]) ] , fastmath = True, nogil = True)
# def sigmaV(X):
#     S, V = np_eigh(X)
#     S[S < 0] = 0.0
#     S **= 0.5

#     S = S[::-1]
#     VT = V[:,::-1].T

#     return S, VT


@njit( [ Tuple(f8[:], f8[:,:])(f8[:,:]), Tuple(f4[:], f4[:,:])(f4[:,:]) ] , fastmath = True, nogil = True)
def eigh_pinv(X):
    S2, V = np_eigh(X)
    return S2, V

In [381]:
# %%time
XT = X.T
XTX = XT @ X
XTX.flat[::XTX.shape[0]+1] += 0.01
cho = linalg.cholesky(  XTX  , check_finite = False )

from scipy.linalg.lapack import dtrtri, strtri
_cho = dtrtri(cho)[0]
_XTX = _cho @ _cho.T
inv = _XTX @ XT
(inv @ X).sum()

206.06060861065558

In [376]:
X.shape

(100000, 200)

In [343]:
## %%time
XT = X.T
XTX = XT @ X
XTX.flat[::XTX.shape[0]+1] += 0.001
S2, V = eigh(XTX + np.eye(p))
cond = np.finfo(X.dtype).eps**2*S2[-1]
# S2[S2 < cond] = cond
V /= S2
inv = (V @ V.T) @ XT
VT = V[:,::-1].T
S2 = S2[::-1]
V = VT.T
cond = S2 < 0
S2[cond] = 1
S = S2 ** 0.5
S2[cond] = np.finfo(X.dtype).eps*S[0]

# U = (X @ VT.T) / S
# pseudo = (V / S) @ U.T
print(np.abs(inv @ X).round(3).sum())
print(np.abs(inv @ X).diagonal().round(3).sum())
print(squareSum(y - X @ (inv @ y))/n)
print(np.square(np.eye(p) - inv @ X).sum())

0.0
0.0
217190.66715885705
199.99681909288105


In [37]:
%%time
XTX = X.T @ X
inv = linalg.eigh(XTX, b = np.eye(len(XTX), dtype = X.dtype), turbo = True,
                 check_finite = False)

Wall time: 2.29 s


In [38]:
%%time
XTX = X.T @ X
inv = eigh_pinv(XTX)

Wall time: 2.02 s


In [ ]:
%%time
inv = ___pinv(X)
print(np.abs(inv @ X).round(3).sum())
print(np.abs(inv @ X).diagonal().round(3).sum())
print(squareSum(y - X @ (inv @ y))/n)
print(np.square(np.eye(p) - inv @ X).sum())

In [ ]:
%%time
inv = linalg.pinv2(X, check_finite = False, return_rank = False)
print(np.abs(inv @ X).round(3).sum())
print(np.abs(inv @ X).diagonal().round(3).sum())
print(squareSum(y - X @ (inv @ y))/n)
print(np.square(np.eye(p) - inv @ X).sum())

In [ ]:
# %%time
import numpy as np
X = np.array([[1,2,np.nan,4],
         [5,np.nan,7,8],
         [9,10,np.nan,np.nan]]).T
mask = np.isnan(X)
X[mask] = 0
XTX = X.T @ X
row_mask = np.ones(len(XTX), dtype = bool)
col_mask = np.ones(len(XTX), dtype = bool)

row_mask[0] = 0
col_mask.ravel()[0] = 0
locate = np.ones((len(XTX), len(XTX)), dtype = bool)
locate[0] = 0
locate[:,0] = 0
XTX[locate].reshape((len(XTX)-1,len(XTX)-1))

In [ ]:
mask[:,0]

In [ ]:
means = X.mean(0)
U2, S22, VT2 = linalg.svd(X, full_matrices = False)
# from sklearn.utils.extmath import svd_flip
# U2, VT2 = svd_flip(U2, VT2)

In [ ]:
((((X @ VT.T) / S) * S) @ VT).round(3)

In [ ]:
((U2 * S2) @ VT2).round(3)

In [ ]:
X.astype(int)

In [ ]:
final = (U * S) @ VT

In [ ]:
S.astype(int)

In [ ]:
S2.astype(int)

In [ ]:
%%time
XXT = X @ X.T
XXT.flat[::XXT.shape[0]+1] += 0.001
chol = linalg.cho_factor(  XXT  , check_finite = False)
t = X.T @ linalg.cho_solve(chol, y)

In [19]:
%%time
XT = X.T
XTX = XT @ X
# XTX.flat[::XTX.shape[0]+1] 
cho = linalg.cholesky(  XTX + np.eye(p)*0.001  , check_finite = False )
t = linalg.cho_solve((cho, False), XT @ y)
print(squareSum(y - X @ t))

4453757746.687452
Wall time: 251 ms


In [296]:
#%%time
XT = X.T
XTX = XT @ X
# XTX.flat[::XTX.shape[0]+1] 
cho = linalg.cholesky(  XTX + np.eye(p)*0.001  , check_finite = False )
from scipy.linalg.lapack import dtrtri, strtri
_cho = dtrtri(cho)[0]
_XTXa = _cho @ _cho.T
t = _XTXa @ (XT @ y)
sigma2 = squareSum(y - X @ t)
sigma = sigma2 ** 0.5

_XTXaXTX = _XTXa @ XTX
exp_value = _XTXaXTX.diagonal()
variance = np.einsum('ij,ij->i', _XTXaXTX, _XTXa)
cond = variance < 0
variance[cond] = variance[~cond].max() * p

In [306]:
np.einsum('ij,ij->i', U, U).sum()

199.99648

In [301]:
(_XTXaXTX @ _XTXa).diagonal()

array([ 1.00487580e-05,  1.00161074e-05,  1.12787099e-05, -1.04842211e-05,
        1.48928820e-05,  1.00634933e-05,  9.98782161e-06,  1.00052212e-05,
        1.00129233e-05,  9.96957780e-06, -6.65776243e-08, -2.91711392e-07,
        1.60480360e-07, -6.51540595e-07, -2.55151228e-07, -4.34103714e-07,
       -1.16338318e-06, -4.41800586e-07, -4.14861511e-07,  2.02813210e-07,
       -4.39876355e-07, -1.68561305e-07, -1.93576166e-07, -4.53345913e-07,
        1.85495209e-07, -1.37773788e-07,  2.77857788e-07,  9.89053121e-08,
       -5.69565294e-08, -2.55151217e-07, -3.78301304e-07, -1.07752337e-08,
        1.08526423e-07,  1.77798338e-07,  1.89343660e-07, -5.11838701e-08,
        1.08526416e-07,  3.85614128e-07,  4.20250107e-07, -2.76317636e-07,
       -3.28271567e-07, -2.22439455e-07, -4.14861482e-07, -7.65069579e-07,
       -8.47811058e-07, -7.51600047e-07, -1.07486904e-06, -1.29230591e-06,
       -1.05177806e-06, -2.58345693e-06,  9.97577870e-06,  9.90624514e-06,
        9.98797963e-06,  

In [204]:
U, S, VT = linalg.svd(X, full_matrices = False, check_finite = False)

In [297]:
pd.concat([
    pd.Series(exp_value*t + sigma*1*(variance**0.5)),
    pd.Series(variance),
    pd.Series(exp_value),
    pd.Series(np.rint(exp*theta + sigma*1*(var ** 0.5))),
    pd.Series(t)
], 1).sort_values(1).round(5)

,0,1,2,3,4,5
27,20.98770,0.00000,945.45456,0.02479,12412.000000,-0.01622
36,21.98483,0.00000,963.41464,0.02479,9939.000000,-0.01622
32,21.98483,0.00000,957.14287,0.02479,10141.000000,-0.01622
12,26.73423,0.00000,700.00001,0.02479,27164.001953,-0.01622
33,28.13980,0.00000,958.90412,0.02479,18950.000000,-0.01622
24,28.74244,0.00000,934.78262,0.02479,9956.000000,-0.01622
34,29.03907,0.00000,960.52633,0.02479,9876.000000,-0.01622
19,30.05423,0.00000,903.22582,0.02479,10345.000000,-0.01622
26,35.17788,0.00000,942.30770,0.02479,10469.000000,-0.01622
37,41.44151,0.00000,964.70589,0.02479,10481.000000,-0.01622


In [198]:
import pandas as pd

In [218]:
exp = ((VT.T * (S**2 / (S**2 + 0.001))) @ VT).diagonal()
var = ((VT.T * (S**2 / (S**2 + 0.001)**2)) @ VT).diagonal()
theta = (VT.T * (S / (S**2 + 0.001))) @ (U.T @ y)

sigma2 = squareSum(y - X @ theta)
sigma = sigma2 ** 0.5



In [119]:
#%%time
inv = np.linalg.inv(XTX + np.eye(p)*0.01)
diagr = (inv @ XTX @ inv).diagonal()
cond = diagr <= 0
mins = diagr[cond].min()
diagr[cond] = mins
diagr

ValueError: assignment destination is read-only

In [ ]:
%%time
inv = linalg.eigh(X @ X.T, check_finite = False)

In [23]:
%%time
XT = X.T
XTX = XT @ X
from torch import potrf as cholesky, potrs as cholesky_triangular_solve
chol = cholesky(  Tensor(XTX)  )
t = cholesky_triangular_solve( Tensor(XT) @ Tensor(y), chol).flatten().numpy()
print(squareSum(y - X @ t))

0.074567124
Wall time: 481 ms


In [ ]:
def pinvv(X):
    U, S, VT = linalg.svd(X, full_matrices = False, check_finite = False)
    cond =  S < np.finfo(X.dtype).eps*S[0] 
    S = np.divide(1.0, S)
    S[cond] = 0.0
    VT *= S.reshape(-1,1)
    return VT.T @ U.T

In [ ]:
%%time
t = np.linalg.eigh(XXT)

In [ ]:
squareSum(y - X @ t[0])

In [ ]:
%%time
XXT = X @ X.T
# XTX = X.T @ X

In [ ]:
%%time
from sklearn.linear_model import Ridge
model = Ridge(fit_intercept = False, alpha = 1, solver = 'cholesky')
model.fit(X, y)
# print(squareSum(y - model.predict(X)))

In [ ]:
from scipy import linalg 
from scipy.linalg import lapack
U = linalg.cholesky(XTX, lower = False, check_finite = False)

In [ ]:
X_U = X @ lapack.strtri(U)[0]

In [ ]:
(X_U * X_U).sum(1)

In [ ]:
%%time
_U = linalg.cho_solve((U, False), np.eye(3))

In [ ]:
_U = lapack.dtrtri(U)[0]
(_U * _U).sum(1)

In [ ]:
import torch
torch.__version__
torch.set_num_threads(4)

In [ ]:
%%time
j = torch.svd(Tensor(X))

In [ ]:
%%time

U, S, VT = torch.svd(Tensor(X))
cond = S < eps(X)*constant(S[0])
_S = 1.0 / S
_S[cond] = 0.0
VT *= _S.reshape(-1,1)
UT = U.numpy().T
inv = VT.numpy().T.dot(UT)

In [ ]:
%%time
inv = torch.pinverse(Tensor(X))

In [ ]:
%%time
t = torch.gels(Tensor(y).reshape(-1,1), Tensor(X))

In [ ]:
%%time
XTX = X.T.dot(X)

In [ ]:
%%time
# x = Tensor(X)
XTX = x.t().matmul(x)

In [ ]:
%%time
a = torch.mm(x.t(), x)

In [ ]:
%%time
from scipy import linalg
a = linalg.pinv2(XTX)

In [ ]:
import tensorflow as tf
tf.enable_eager_execution()

In [ ]:
%%time
X.T @ X

In [ ]:
%%time
x = Tensor(XT)
x @ Tensor(X)

In [ ]:
%%time
XT = X.T
XTX = XT.dot(X)
# theta_hat = pinv(XTX).dot(XT.dot(y))
# print(((y - X.dot(theta_hat))**2).sum())

In [ ]:
%%time
a = x.numpy()
a.T.dot(a)

In [ ]:
np.dot

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.linalg as linalg

In [ ]:
data = pd.read_table('C:/Users/danie/Downloads/overdue.txt')
data.columns
data['isResid'] = 0
data['isComm'] = 0
data.loc[:48, 'isResid'] = 1
data.loc[48:, 'isComm'] = 1
data['Residential'] = data['BILL']*data['isResid']
data['Commercial'] = data['BILL']*data['isComm']
data['Bias'] = 1
y = data.pop('LATE')
data.pop('BILL');

X = data.values
thetas = linalg.pinv2(X.T.dot(X)).dot( X.T.dot(y)  )
y_hat = X.dot(thetas)

e_hat = y - y_hat
e_hat

In [ ]:
plt.scatter(y, e_hat)

In [ ]:
plt.scatter(residential['BILL'], residential['LATE'])

In [ ]:
%%time
inv = pinv(XTX)

In [ ]:
import torch
torch.set_num_threads(4)

In [ ]:
%%time
a = svd(X)

In [ ]:
%%time
theta_hat = svd_solve(X, y)
print(((y - X.dot(theta_hat))**2).sum())

In [ ]:
%%time
cholesky_solve(X, y)

In [ ]:
((y - X.dot(svd_solve(X, y)))**2).sum()

In [ ]:
from scipy.linalg.lapack import clapack
Tensor(clapack.dtrtri(R)[0])

In [ ]:
from torch import trtrs

In [ ]:
print_all_warnings = True

from torch import potrf as cholesky_decomposition, diag, ones, \
                potrs as cholesky_triangular_solve
from numpy import round as np_round

def t_cholesky_solve(X, y, alpha = 0, step = 2):
    '''
    Solve least squares problem X*theta_hat = y using Cholesky Decomposition.
    
    Alpha = 0, Step = 2 can be options
    Alpha is Regularization Term and step = 2 default for guaranteed stability.
    Step must be > 1
    
    |  Method   |   Operations    | Factor * np^2 |
    |-----------|-----------------|---------------|
    | Cholesky  |   1/3 * np^2    |      1/3      |
    |    QR     |   p^3/3 + np^2  |   1 - p/3n    |
    |    SVD    |   p^3   + np^2  |    1 - p/n    |
    
    NOTE: HyperLearn's implementation of Cholesky Solve uses L2 Regularization to enforce stability.
    Cholesky is known to fail on ill-conditioned problems, so adding L2 penalties helpes it.
    
    Note, the algorithm in this implementation is as follows:
    
        alpha = dtype(X).decimal    [1e-6 is float32]
        while failure {
            solve cholesky ( XTX + alpha*identity )
            alpha *= step (2 default)
        }
    
    If MSE (Mean Squared Error) is abnormally high, it might be better to solve using stabler but
    slower methods like qr_solve, svd_solve or lstsq.
    
    https://www.quora.com/Is-it-better-to-do-QR-Cholesky-or-SVD-for-solving-least-squares-estimate-and-why
    '''
    assert step > 1
    
    XTX = T(X).matmul(X)
    regularizer = ones(X.shape[1]).type(X.dtype)
    
    if alpha == 0: 
        alpha = typeTensor([np_finfo(dtype(X)).resolution]).type(X.dtype)
    no_success = True
    warn = False

    while no_success:
        alphaI = regularizer*alpha
        try:
            chol = cholesky_decomposition(  XTX + diag(alphaI)  )
            no_success = False
        except:
            alpha *= step
            warn = True
            
    if warn and print_all_warnings:
        addon = np_round(constant(alpha), 10)
        print(f'''
            Matrix is not full rank. Added regularization = {addon} to combat this. 
            Now, solving L2 regularized (XTX+{addon}*I)^-1(XTy).

            NOTE: It might be better to use svd_solve, qr_solve or lstsq if MSE is high.
            ''')
   
    XTy = T(X).matmul( ravel(y, chol)  )
    
    theta_hat = cholesky_triangular_solve(XTy, chol).flatten()
    return theta_hat

cholesky_solve = n2n(t_cholesky_solve)
_cholesky_solve = n2t(t_cholesky_solve)

In [ ]:
%%time
theta_hat = cholesky_solve(X, y)

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [ ]:
%%time
model.fit(X, y)

In [ ]:
preds = X.dot(theta_hat)
e_hat = y - preds
MSE = (e_hat**2).sum() / (n-p)
MSE

In [ ]:
preds = X.dot(theta_hat_svd)
e_hat = y - preds
MSE = (e_hat**2).sum() / (n-p)
MSE

In [ ]:
np.square(Yx - X.dot(t)).sum()

In [ ]:
%%time
t = _lstsq(X, Yx)[0]

In [ ]:
XX = toTensor(X)

In [ ]:
%%time
XTX = XX.t().matmul(XX)
XTXt = X.T.dot(X)

In [ ]:
J = tf.convert_to_tensor(X)

In [ ]:
%%time
XTX = tf.matmul(X, X, adjoint_a = True)

In [ ]:
svd_solve(X, y.astype(np.int))

In [ ]:
chol

In [ ]:
inverse = torch.potri(chol)

In [ ]:
%%time
theta = torch.potrs(XTy, chol)
XX.matmul(theta)

In [ ]:
%%time
cholt = tf.linalg.cholesky(XTXt)
XTy = X.T.dot(Ys.reshape(-1,1))

In [ ]:
%%time
theta = tf.linalg.cholesky_solve(cholt, XTy)
X.dot(theta)

In [ ]:
%%time
tf.linalg.lstsq(X, Yx.reshape(-1,1).astype(np.float32),
                    l2_regularizer = 1.0/10000)

In [ ]:
((Yx - X.dot(T).ravel())**2).sum()

In [ ]:
from hyperlearn.temp import addition

In [ ]:
Parallel(addition)([1,5,1], [2,2,2], [3,3,3])

In [ ]:
finalOutput

In [ ]:
(X == 0).dtype is np.bool_

In [ ]:
%%time
from hyperlearn.discriminant_analysis import QuadraticDiscriminantAnalysis
model = QuadraticDiscriminantAnalysis(n_jobs = -1)
model.fit(X, y)

In [ ]:
%%time
model.predict(X)

In [ ]:
%%time
j = X - model.means_[0]
out = toTensor(j).matmul(model.scaled_rotations_[0])

In [ ]:
%%time
mo

In [ ]:
%%time
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
model = QuadraticDiscriminantAnalysis()
model.fit(X, y)

In [ ]:
%%time
model.score(X, y)

In [ ]:
%%time
alpha = 0.001
a_1 = 1-alpha

class_priors = y.bincount().type(torch.float32) / X.shape[0]
classes = y.unique()

class_scalings = []; class_rotations = []; class_means = []; log_scalings = []

for x in classes:
    partial_X = X[y == x]
    partial_mean = partial_X.mean(0)
    partial_X -= partial_mean
    
    U, S, VT = _svd(partial_X)
    V = T(VT)
    scale = (S**2) / (partial_X.shape[0] -1)
    scale = alpha + (a_1 * scale)
    
    #partial_cov = (V * scale).matmul(VT)
    
    class_scalings.append(scale)
    log_scalings.append(scale.log().sum())
    class_rotations.append(V)
    class_means.append(partial_mean)
    
class_log_scalings = stack(log_scalings)
class_log_priors = class_priors.log()

In [ ]:
%%time
distances = []

for V, S, means in zip(class_rotations, class_scalings, class_means):
    partial_X = (X - means).matmul(   V/S**0.5   )
    
    plt.scatter(partial_X[:,0], partial_X[:,1], c = y, cmap = 'magma', alpha = 0.3)
    plt.show()
    distances.append(  einsum('ij,ij->i', partial_X, partial_X) )
    #distances.append(  (partial_X**2).sum(1)  )

distances = T(stack(distances))
decision = -0.5 * (distances + class_log_scalings) + class_log_priors

likelihood = (decision - T(decision.max(1)[0])).exp()
sum_softmax = T( einsum('ij->i', likelihood) )
#sum_softmax = T(likelihood.sum(1))
softmax = likelihood / sum_softmax

pred = classes.take(softmax.argmax(1)).numpy()
true = y.numpy()
print((pred == true).sum() / len(X) * 100)

In [ ]:
-0.5*(distances + class_log_scalings) + class_log_priors

In [ ]:
distances

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(10, input_shape = (X.shape[1], ), activation = 'relu') )
model.add(Dense(5, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc'])
model.fit(X.numpy(), labels.type(torch.float).numpy(), epochs = 10, batch_size = 128)

In [ ]:
from torch import nn, optim

model = nn.Sequential(
    nn.Linear(X.shape[1], 5),
#     nn.ReLU(),
#     nn.Linear(100, 5)
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), weight_decay = 0.001, lr = 0.05)

labels = []
for i in y.unique():
    labels.append(y==i)
labels = T(stack(labels)).type(torch.LongTensor )

In [ ]:
ys = y.type(torch.long)

In [ ]:
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, ys)
    loss.backward()
    optimizer.step()
    print((model(X).argmax(1) == ys).sum().type(torch.float) / len(X))

In [ ]:
vars(model[0])

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(n_jobs = -1, multi_class = 'ovr')
model.fit(X, y)

In [ ]:
LogisticRegression()

In [ ]:
%%time
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
model = QuadraticDiscriminantAnalysis(reg_param = 0.01)
model.fit(X, y)

In [ ]:
400/2700

In [ ]:
%%time
pred = model.predict(X)

In [ ]:
%%time
(pred == y).sum().item() / len(X)

In [ ]:
class_scalings[0].numpy().round(1), model.scalings_[0].round(1)

In [ ]:
%%time
size = len(X)
x = torch.randperm(size)

x_train, x_test = X[x].chunk(2)
y_train, y_test = y[x].chunk(2)
size = len(x_test)

y_true = y_test.numpy()
class_priors = y_train.bincount().type(torch.float32) / x_train.shape[0]
class_log_priors = class_priors.log()

classes = y_train.unique()

scalings = []
rotations = []
means = []
log_scalings = []

for x in classes:
    partial_X = x_train[y_train == x, :]
    partial_mean = partial_X.mean(0)
    partial_X -= partial_mean
    
    U, S, VT = _svd(partial_X)
    V = T(VT)
    scale = (S**2) / (partial_X.shape[0] -1)
    
    scalings.append(scale)
    rotations.append(V)
    means.append(partial_mean)

    
scores = []
alphas = np.arange(0.001, 1, 0.1)

partials = []
for x in class_means:
    partials.append( x_test - x )
    
for alpha in alphas:
    log_scalings = []
    distances = []
    
    a_1 = 1 - alpha
    
    for partial_X, V, S in zip(partials, rotations, scalings):
        scale = alpha + (a_1 * S)
        log_scalings.append(scale.log().sum())
        
        partial_X = partial_X.matmul(   V/(scale**0.5)   )
        
        #distances.append(   (partial_X**2).sum(1)   )
        distances.append(   einsum('ij,ij->i', partial_X, partial_X)   )
    
    class_log_scalings = stack(log_scalings)
    distances = T(stack(distances))
    decision = -0.5 * (distances + class_log_scalings) + class_log_priors

    likelihood = (decision - T(decision.max(1)[0])).exp()
    
    sum_softmax = T(likelihood.sum(1))
    #sum_softmax = T(toTensor(einsum('ij->i', likelihood)))
    softmax = likelihood / sum_softmax

    pred = classes.take(softmax.argmax(1)).numpy()
    scores.append((pred == y_true).sum()/size)
scores = np.array(scores)

In [ ]:
alphas[scores.argmax()]

In [ ]:
plt.plot(alphas, scores)

In [ ]:
pcaed = model.transform(centered)
plt.scatter(x = pcaed[:,0], y = pcaed[:,1], c = y)

In [ ]:
if self.priors is None:
            self.priors_ = np.bincount(y) / float(n_samples)
        else:
            self.priors_ = self.priors

        cov = None
        store_covariance = self.store_covariance or self.store_covariances
        if self.store_covariances:
            warnings.warn("'store_covariances' was renamed to store_covariance"
                          " in version 0.19 and will be removed in 0.21.",
                          DeprecationWarning)
        if store_covariance:
            cov = []
        means = []
        scalings = []
        rotations = []
        for ind in xrange(n_classes):
            Xg = X[y == ind, :]
            meang = Xg.mean(0)
            means.append(meang)
            if len(Xg) == 1:
                raise ValueError('y has only 1 sample in class %s, covariance '
                                 'is ill defined.' % str(self.classes_[ind]))
            Xgc = Xg - meang
            # Xgc = U * S * V.T
            U, S, Vt = np.linalg.svd(Xgc, full_matrices=False)
            rank = np.sum(S > self.tol)
            if rank < n_features:
                warnings.warn("Variables are collinear")
            S2 = (S ** 2) / (len(Xg) - 1)
            S2 = ((1 - self.reg_param) * S2) + self.reg_param
            if self.store_covariance or store_covariance:
                # cov = V * (S^2 / (n-1)) * V.T
                cov.append(np.dot(S2 * Vt.T, Vt))
            scalings.append(S2)
            rotations.append(Vt.T)
        if self.store_covariance or store_covariance:
            self.covariance_ = cov
        self.means_ = np.asarray(means)
        self.scalings_ = scalings
        self.rotations_ = rotations
        return self

In [ ]:
U, S, VT = _svd(X)
Q, R = _qr(X)

In [ ]:
%%time
ridge_stats(U, S, VT)

In [ ]:
ridge_stats(U, S, VT)

In [ ]:
einsum('ij,ij->i', (U, U) )

In [ ]:
np.einsum('ij,ij->i', UU, UU)

In [ ]:
UU, SS, VTVT = svd(X)

In [ ]:
%%timeit -n3 -r1
U, S, VT = svd(X)
U = S = VT = None
gc.collect()

In [ ]:
%%timeit  -n3 -r1
Q, R = qr(X)
Q = R = None
gc.collect()

In [ ]:
u, s, vt =_svd(X)

In [ ]:
%%time
u.matmul(torch.diag(s))

In [ ]:
    max_U = np_argmax(np_abs(U), axis = 0)
    signs = np_sign(U[max_U, range(U.shape[1])])
    U *= signs
    VT *= signs[:, np_newaxis]

In [ ]:
%%time
_pinv(Tensor(X))

In [ ]:
    U, S, VT = np_svd(X, full_matrices = False)
    cond =  S < np_finfo(X.dtype).eps*S[0] 
    S = np_divide(1.0, S)
    S[cond] = 0.0
    VT *= S.reshape(-1,1)
    return np_dot(VT.T, U.T)

In [ ]:
%%time


In [ ]:
%%time
_pinv(X)

In [ ]:
torch.diag(_S)

In [ ]:
import numpy as np

X = np.random.random((100000,100))
X = X.astype(dtype = np.float32)
# X = Tensor(X)

In [ ]:
%%time
X = Tensor(X)

In [ ]:
%%timeit
svd(X, some = True)

In [ ]:
with tf.Session() as sess:
    sess.run(init)
tf.matmul( tf.multiply(U, S) , V, transpose_b = True)

In [ ]:
Xt = nn.Tensor(X)

In [ ]:
%%time
U, S, V = nn.svd(Xt)

In [ ]:
tf.transpose(V)

In [ ]:
R

In [ ]:
%%timeit -n3 -r1
U, S, VT = _svd(X)
U = S = VT = None
gc.collect()

In [ ]:
%%timeit -n3 -r1
Q, R = _qr(X)
Q = R = None
gc.collect()